<a href="https://colab.research.google.com/github/ev1025/CodeStates-Project2/blob/main/%EC%98%88%EB%A7%A4%EC%88%98%EC%9A%94%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 목표(기업)

### 결과 및 측정 데이터 분석 및 예매 예측

1. 지난 경기 관람 만족도 연계 다음경기 예매 예측
   - 이겼다
   - 이겼지만 마음에 들지 않음
   - 무승부
   - 졌지만 잘싸웠다
   - 졌다   
2. 관람여부와 경기결과 및 순위에 따른 예매 예측(+고급좌석 예매 예측)
   - **관람**
      - 승리
        - 상위
        - 중위
        - 하위
      - 패배
        - 상위
        - 중위
        - 하위
   - **비관람**
      - 승리
        - 상위
        - 중위
        - 하위
      - 패배
        - 상위
        - 중위
        - 하위
     

### 예매 촉진 지표 분석 및 적정 프로모션 수준 예측

1. 경기 결과 및 순위에 따른 가격 인상,할인,프로모션 진행의 실효성
   - 가격 인상
   - 가격 할인
   - 프로모션
2. 경기 관람 만족도에 따른 분석 및 예측 모델링
   - 만족도
      - 경기장 시설(청결도, 온도조절)
      - 경기장 운영 
      - 경기내용
      - 음식의 맛
      - 관람료

### 예매 촉진을 위한 프로모션 대상자 세분화 및 촉진 방법 제안

1. 고객 세분화 및 액션아이템 고안
  - 고객 세분화
    - 적절한 할인율
    - 적절한 프로모션
    - 고급좌석이용고객 별도 세분화

### 가설

- 지난경기의 관람 여부, 순위, 경기결과에 따라 예매 수가 다를 것이다.
- 위의 가설에 해당하지 않는 고객군이 존재 할 것이다.
- 예매유도에 반응하는 고객군이 있을 것이다.
- 고급좌석 이용을 예측할 수 있다.

# 나의 생각

### 기업에서 원하는 것

- 관람여부, 경기결과, 순위가 예매에 미치는 영향 분석 및 모델링
- 마케팅에 반응하는 고객층을 찾아서 수익 극대화
- 고급좌석 이용률을 높이는 방법에 대한 고민

### 가설

- 일반팬과 골수팬을 분류할 수 있다.
- 경기의 승패는 예매와 관련이 없을 것이다.
- 관람이 뜸한 충성고객을 찾아서 다시 불러들일 수 있다.

### 프로모션(아이디어)

- 질 것 같은 경기일 수록 응원의 힘으로 승리를 이끌어야 한다.   
=> 전력상 패배가 예상되는 팀의 좌석을 예매한 고객에 한해서 그 경기가 승리할 경우 고급좌석 예매 할인 쿠폰 제공

- 승부예측    
경기의 스코어를 맞추는 고객에 한하여 쿠폰지급



# 데이터

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/더케이/membership.csv')
df2 = pd.read_csv('/content/drive/MyDrive/더케이/reserve_ticket_.csv')
df3 = pd.read_csv('/content/drive/MyDrive/더케이/survey_complete_.csv')
df4 = pd.read_csv('/content/drive/MyDrive/더케이/survey_item.csv')
df5 = pd.read_csv('/content/drive/MyDrive/더케이/survey_main.csv')
df6 = pd.read_csv('/content/drive/MyDrive/더케이/survey_result.csv')
df7 = pd.read_excel('/content/drive/MyDrive/더케이/d95e264436cf42de.xlsx',engine = 'openpyxl') # excel 여는법

In [18]:
# xml 여는법 다시 찾아보기
# 인코딩 깨질 때는 encode -> decode 다시 할 것
import xml.etree.ElementTree as ET
tree = ET.parse('/content/drive/MyDrive/더케이/schedule2022120221019212245.xml', parser=ET.XMLParser(encoding='latin1')) # multi encode오류 날 때 'latin-1',  'iso-8859-5', 'cp1252'
root = tree.getroot()

df8 = []
for i in root: # i.tag , i.attrib 알아보기
    i.attrib['yoil'] = i.attrib['yoil'].encode('latin-1').decode('euc-kr')
    i.attrib['meet_name'] = i.attrib['meet_name'].encode('latin-1').decode('euc-kr')
    i.attrib['home_team_name'] = i.attrib['home_team_name'].encode('latin-1').decode('euc-kr')
    i.attrib['away_team_name'] = i.attrib['away_team_name'].encode('latin-1').decode('euc-kr')
    i.attrib['game_type_name'] = i.attrib['game_type_name'].encode('latin-1').decode('euc-kr')
    i.attrib['field_name'] = i.attrib['field_name'].encode('latin-1').decode('euc-kr')
    df8.append(i.attrib)

In [ ]:
# 리스트 안의 dict[{}]를 데이터프레임으로 만들기
empty_df = pd.DataFrame() # 빈 데이터프레임 만들기
for i in df8:
    i = pd.DataFrame.from_dict(i, orient='index').T # dict를 dataframe으로 만들기
    empty_df = empty_df.append(i)                   # 각 dataframe을 빈데이터프레임에 추가
empty_df.reset_index(drop=True)

In [20]:
empty_df.to_csv('/content/drive/MyDrive/더케이/schedule.csv', index=False)

In [ ]:
a = pd.read_csv('/content/drive/MyDrive/더케이/schedule.csv')